# 📄 Descarga Simple de Facturas de Ejemplo

Este notebook descarga facturas de ejemplo desde URLs públicas para probar el sistema de extracción.

**Ventajas de este método:**
- ✅ No requiere dependencias complejas
- ✅ Descarga directa desde URLs
- ✅ Compatible con cualquier entorno Python

In [ ]:
# Instalar solo las dependencias básicas necesarias
%pip install requests pillow reportlab

In [ ]:
import os
import requests
from pathlib import Path
from PIL import Image
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import json

In [ ]:
# Crear directorio para guardar las facturas
facturas_dir = Path('../facturas_dataset')
facturas_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorio creado: {facturas_dir.absolute()}")
print(f"📁 Las facturas se guardarán aquí")

In [ ]:
# URLs de facturas de ejemplo (imágenes públicas de facturas)
# Estas son URLs de ejemplo - puedes reemplazarlas con tus propias URLs
facturas_urls = [
    {
        "url": "https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/test/eurotext.png",
        "nombre": "factura_ejemplo_1",
        "tipo": "imagen"
    },
]

print(f"📥 Preparando descarga de {len(facturas_urls)} facturas de ejemplo...")

In [ ]:
# Función para descargar una imagen
def descargar_imagen(url, nombre_archivo):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Guardar la imagen
        img_path = facturas_dir / f"{nombre_archivo}.png"
        with open(img_path, 'wb') as f:
            f.write(response.content)
        
        return img_path, True
    except Exception as e:
        print(f"❌ Error descargando {nombre_archivo}: {e}")
        return None, False

# Función para convertir imagen a PDF
def imagen_a_pdf(img_path, pdf_path):
    try:
        # Abrir la imagen
        img = Image.open(img_path)
        
        # Crear PDF
        c = canvas.Canvas(str(pdf_path), pagesize=letter)
        
        # Calcular dimensiones para ajustar a la página
        img_width, img_height = img.size
        aspect = img_height / float(img_width)
        
        page_width, page_height = letter
        display_width = page_width - 40
        display_height = display_width * aspect
        
        if display_height > page_height - 40:
            display_height = page_height - 40
            display_width = display_height / aspect
        
        # Dibujar imagen en el PDF
        c.drawImage(str(img_path), 20, page_height - display_height - 20, 
                   width=display_width, height=display_height)
        c.save()
        
        return True
    except Exception as e:
        print(f"❌ Error convirtiendo a PDF: {e}")
        return False

In [ ]:
# Descargar facturas
facturas_descargadas = 0

for factura in facturas_urls:
    print(f"\n📥 Descargando: {factura['nombre']}...")
    
    img_path, success = descargar_imagen(factura['url'], factura['nombre'])
    
    if success:
        print(f"✅ Imagen guardada: {img_path.name}")
        
        # Convertir a PDF
        pdf_path = facturas_dir / f"{factura['nombre']}.pdf"
        if imagen_a_pdf(img_path, pdf_path):
            print(f"✅ PDF creado: {pdf_path.name}")
            facturas_descargadas += 1
        
        # Crear metadata
        metadata = {
            "nombre": factura['nombre'],
            "url_origen": factura['url'],
            "tipo": factura['tipo'],
            "archivos": {
                "imagen": f"{factura['nombre']}.png",
                "pdf": f"{factura['nombre']}.pdf"
            }
        }
        
        metadata_path = facturas_dir / f"{factura['nombre']}_metadata.json"
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\n🎉 Proceso completado!")
print(f"✅ {facturas_descargadas} facturas descargadas y convertidas a PDF")
print(f"📁 Ubicación: {facturas_dir.absolute()}")

## 🔧 Crear Facturas de Ejemplo Personalizadas

Si no tienes URLs de facturas, puedes crear facturas de ejemplo con datos ficticios:

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from datetime import datetime

def crear_factura_ejemplo(numero, nombre_archivo):
    """Crea una factura de ejemplo con datos ficticios"""
    pdf_path = facturas_dir / nombre_archivo
    doc = SimpleDocTemplate(str(pdf_path), pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()
    
    # Título
    title = Paragraph(f"<b>FACTURA #{numero:04d}</b>", styles['Title'])
    elements.append(title)
    elements.append(Spacer(1, 0.3*inch))
    
    # Información del proveedor
    proveedor_data = [
        ['Proveedor:', 'Empresa Ejemplo S.A. de C.V.'],
        ['RFC:', 'EEJ123456ABC'],
        ['Dirección:', 'Av. Principal #123, Ciudad, CP 12345'],
        ['Teléfono:', '(55) 1234-5678'],
    ]
    
    proveedor_table = Table(proveedor_data, colWidths=[1.5*inch, 4*inch])
    proveedor_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ]))
    elements.append(proveedor_table)
    elements.append(Spacer(1, 0.3*inch))
    
    # Información del cliente
    cliente_data = [
        ['Cliente:', 'Cliente Ejemplo S.A.'],
        ['RFC:', 'CEJ987654XYZ'],
        ['Fecha:', datetime.now().strftime('%d/%m/%Y')],
    ]
    
    cliente_table = Table(cliente_data, colWidths=[1.5*inch, 4*inch])
    cliente_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ]))
    elements.append(cliente_table)
    elements.append(Spacer(1, 0.3*inch))
    
    # Items
    items_data = [
        ['Cantidad', 'Descripción', 'Precio Unit.', 'Total'],
        ['2', 'Producto A', '$500.00', '$1,000.00'],
        ['1', 'Servicio B', '$1,500.00', '$1,500.00'],
        ['3', 'Producto C', '$250.00', '$750.00'],
    ]
    
    items_table = Table(items_data, colWidths=[1*inch, 3*inch, 1.2*inch, 1.2*inch])
    items_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(items_table)
    elements.append(Spacer(1, 0.3*inch))
    
    # Totales
    totales_data = [
        ['Subtotal:', '$3,250.00'],
        ['IVA (16%):', '$520.00'],
        ['Total:', '$3,770.00'],
    ]
    
    totales_table = Table(totales_data, colWidths=[4.5*inch, 1.5*inch])
    totales_table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'RIGHT'),
        ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 11),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('LINEABOVE', (0, -1), (-1, -1), 2, colors.black),
    ]))
    elements.append(totales_table)
    
    # Generar PDF
    doc.build(elements)
    return pdf_path

# Crear 5 facturas de ejemplo
print("📝 Creando facturas de ejemplo personalizadas...\n")

for i in range(1, 6):
    nombre_archivo = f"factura_ejemplo_{i}.pdf"
    pdf_path = crear_factura_ejemplo(i, nombre_archivo)
    print(f"✅ Creada: {pdf_path.name}")
    
    # Crear metadata
    metadata = {
        "numero_factura": f"{i:04d}",
        "tipo": "factura_generada",
        "fecha_creacion": datetime.now().isoformat(),
        "datos_ejemplo": {
            "proveedor": "Empresa Ejemplo S.A. de C.V.",
            "rfc_proveedor": "EEJ123456ABC",
            "cliente": "Cliente Ejemplo S.A.",
            "rfc_cliente": "CEJ987654XYZ",
            "subtotal": 3250.00,
            "iva": 520.00,
            "total": 3770.00
        }
    }
    
    metadata_path = facturas_dir / f"factura_ejemplo_{i}_metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\n🎉 5 facturas de ejemplo creadas exitosamente!")
print(f"📁 Ubicación: {facturas_dir.absolute()}")

In [ ]:
# Listar todas las facturas creadas
print("📋 Facturas disponibles:\n")

pdf_files = list(facturas_dir.glob("*.pdf"))
for pdf_file in sorted(pdf_files):
    size_mb = pdf_file.stat().st_size / (1024 * 1024)
    print(f"  📄 {pdf_file.name} ({size_mb:.2f} MB)")

print(f"\n✅ Total: {len(pdf_files)} facturas en formato PDF")
print(f"\n💡 Ahora puedes usar estas facturas en la interfaz web:")
print(f"   1. Ve a http://localhost:3000/login")
print(f"   2. Inicia sesión")
print(f"   3. Carga cualquiera de estas facturas PDF")
print(f"   4. Observa cómo OpenAI extrae los datos automáticamente")